In [3]:
from qiskit.converters import circuit_to_dag, dag_to_circuit
from qiskit.tools.visualization import dag_drawer
from help_fun import cut_edges, generate_sub_circs
import pickle

circ = pickle.load(open('results/supremacy_circuit_4_8.dump', 'rb' ))

In [9]:
original_dag = circuit_to_dag(circ)
dag_drawer(original_dag, filename='results/original_dag.pdf')
circ.draw(output='text',line_length=400,filename='results/original_circ.txt')

q = circ.qregs[0]

positions = [(q[8], 3), (q[9], 5), (q[10], 4), (q[11], 4)]
cut_dag = cut_edges(original_dag=original_dag, positions=positions)
dag_drawer(cut_dag, filename='results/cut_dag.pdf')
# dag_to_circuit(cut_dag).draw(output='mpl',filename='cut_circ.pdf')

# sub_circs, _ = generate_sub_circs(cut_dag, q[2])
# for i, sub_circ in enumerate(sub_circs):
#     dag_drawer(circuit_to_dag(sub_circ), filename='sub_dag_%d.pdf' % i)
#     sub_circ.draw(output='mpl',filename='sub_circ_%d.pdf' % i)
print()